# Dataload Subject specific data

Building_shapes is a generated shapefile, Delivery_data is the data that will augment the map visualisations

In [2]:
Delivery_data = pd.read_csv('toy_delivery_horoshevskiy.csv')
Building_shapes = gpd.read_file('toy_building_choropleths_horoshevskiy.shp')

/home/sam/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,23,38,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Creating and centring an empty Folium Map

Maps can either be displayed in Jupyter simply by calling the object, or can be saved down as demonstrated below

In [3]:
# m = folium.Map(location=[55.764414, 37.647859], tiles='Stamen Toner')
# m
# m.save('test_map.html')

# Building_shapes data transformations

In order to visualise choropleths from the data it is necessary to create a json which describes the physical mapping of the shapes, and a dictionary, which attributes each shape a particular value.

In [4]:
choropleth_json = gpd.GeoDataFrame(Building_shapes).to_json()
Choro_dict = Building_shapes[['delivery_building_id', 'order_total']].set_index('delivery_building_id')['order_total']

# Colourmap for the Choropleths

A colourmap is created. Here I have chosen a gradient from red to black to blue, and have set the limits to the 10th and 90th percentiles of the data. This is to ensure that no outliers heavily skew the colour palette. The colourmap is also captioned with the name of the data column.

In [5]:
linear = cm.LinearColormap(['red','black','blue'], 
                           vmin=Building_shapes['order_total'].quantile(.1),
                           vmax=Building_shapes['order_total'].quantile(.9))

linear.caption = 'Order Total'

# Choropleth generation

The Choropleths are generated using the json previously defined. They are then passed a style function, which uses the colourmap and the dictionary to assign a particular colour to each choropleth

In [6]:
choropleths = folium.GeoJson(
        json.loads(choropleth_json),
        smooth_factor = 1.0,
        style_function=lambda feature: 
        {
        'fillColor': linear(Choro_dict[feature['properties']['delivery_building_id']]),
        'color' : 'black',
        'weight' : 1,
        'dashArray' : '5, 5'
        },
        )

# Adding children to map, and visualisation

Zooming in on the Horishovskiy region, you can see Choropleths of buildings have been added and coloured. These choropleths are non-interactive, but by using markers in the next section, an element of user interactivity can be added. The colourmap can be added simply to the map as a child, in the same way that the choropleths can.

In [7]:
m = folium.Map(location=[55.764414, 37.647859], tiles='Stamen Toner', zoom_start=12)

m.add_child(linear)
m.add_child(choropleths)

m

# Generating Basic Data Markers for Choropleths

The functions below are used to add interactive markers to the maps. Both of these functions should be called iteratively, as seen in the next code block

admin_central_popups: This function retrives the data from each line of the GeoDataFrame that is used for making the contents of the popup. In the case below, I have created a string with linebreaks, to display the id of the building, the order total average and the count for each building. row[1] pulls up a singular row of the geodataframe when called with iterrows.

add_popup: After generating the contents of the popup, it must be added to the graph. This function allows for placing the popup at the centroid of the object shape, with optional offset and colour. Size and shape of the point can be changed in the function.

In [8]:
def admin_central_popups():
    return folium.Popup('Osm ID: ' + str(row[1].delivery_building_id)+ '<br>\
                       Order Total: ' + (str(row[1].order_total)) + '<br>\
                       Count: ' + str(row[1]['count']))
                        
def add_popup(x_offset, y_offset, color):
    folium.RegularPolygonMarker([row[1].geometry.centroid.y + y_offset,
                         row[1].geometry.centroid.x + x_offset],  
                         popup=p,
                         number_of_sides=4,
                         radius=5,
                         fill_color= color).add_to(m)    

# Adding markers to maps

The functions below are called, iterating through the GeoDataFrame shapes. At the end of the add_popup function there is a function to add it to m, demonstrating a different way of adding children to the map.

There is no reason why multiple layers of markers or choropleths cannot be added to a graph. However, most HTML maps fail to open if they contain too many children.

In [9]:
m = folium.Map(location=[55.764414, 37.647859], tiles='Stamen Toner', zoom_start= 12)

for row in Building_shapes.iterrows():   
    p = admin_central_popups()      
    add_popup(x_offset = 0, y_offset = 0, color = 'orange')
    
m

# Filter data for graph construction

Below data is replaced with 'Other' if it is not one of the top_n food categories

In [10]:
filter_list = list(Delivery_data.chain_specialization.value_counts()[:7].index)

def filter_other(string):
    if string in filter_list:
        return string
    else:
        return 'Other'

Delivery_data.chain_specialization = Delivery_data.chain_specialization.apply(lambda x: filter_other(x))

# Arranging Data for Graph Popup

Instead of just placing a string in a popup, a graph can be inserted via vincent. The first two lines of code generate a bargraph database of the entire data (in order to compare each building's data to the overall data in the visualised graph). The data looks like this:

In [11]:
#Generate the overall foodtype_df to be used in all foodtype_dist_popups calls
overall_foodtype_df = pd.DataFrame(Delivery_data.chain_specialization.value_counts())
overall_foodtype_df.chain_specialization = overall_foodtype_df.chain_specialization/overall_foodtype_df.chain_specialization.sum()

overall_foodtype_df

chain_specialization
Пицца                      0.357652
Суши                       0.254915
Шашлыки                    0.110164
Other                      0.108173
Бургеры                    0.102278
Китайская еда              0.043805
Пироги                     0.023013

# Feeding data to Vincent

This data can be fed directly to a vincent bargraph, as shown in the below code. A vincent graph can be represented as a json, which, through Vega can be added as a component in a popup in folium. Below I have demonstrated how the above data is visualised in vincent.

In [12]:
vincent.core.initialize_notebook()

group = vincent.GroupedBar(overall_foodtype_df)
group.colors(brew='Set2')
group.width=400
group.height = 200
group.legend(title='Categories')
group.display()

# Combining the above data with specific data, and adding to popup

The function below takes the above data, and combines it iteratively with all of the shapefiles specified. GroupedBar is the function called, but many different vincent graphs exist.

After each graph is generated, it is converted to json, and that json is fed to folium.Vega, which is in turn added to the popup.

In [13]:
def foodtype_dist_popups():
    data = Delivery_data[Delivery_data['delivery_building_id'] == row[1].delivery_building_id]
    data = data[['chain_specialization', 'delivery_building_id']]
    
    region_df = pd.DataFrame(data.chain_specialization.value_counts())
    region_df.chain_specialization = region_df.chain_specialization/region_df.chain_specialization.sum()
    
    plotting_df = pd.concat([region_df, overall_foodtype_df], axis = 1)
    plotting_df.columns = [row[1].delivery_building_id, 'Overall Restaurant Type Distribution']
    plotting_df = plotting_df.sort_values('Overall Restaurant Type Distribution', ascending=False)
    
    group = vincent.GroupedBar(plotting_df)
    group.colors(brew='Set2')
    group.width=400
    group.height = 200
    group.legend(title='Categories')

    #Convert plot to json to be included in popup
    linejson = group.to_json()

    #Create popup and add chart to popup
    popup = folium.Popup(max_width=550)
    folium.Vega(linejson, height=250, width=550).add_to(popup)
    return popup

# Plotting graphs on datapoints

The graph popups explained above are plotted on the map below. I have included an offset in the new graph markers, alongside the original data markers, as well as the original choropleths introduced at the beginning, and its colourmap.

The data in the map below is restricted to the first 20 building shapes, due to the graphical intensity of generating the map within Notebooks.

In [14]:
m = folium.Map(location=[55.764414, 37.647859], tiles='Stamen Toner', zoom_start= 12)

for row in Building_shapes[:20].iterrows():   

    #Original base data markers
    p = admin_central_popups()      
    add_popup(x_offset = 0, y_offset = 0, color = 'orange')    
    
    #New Graph markers
    p = foodtype_dist_popups()      
    add_popup(x_offset = 0.0001, y_offset = 0.0001, color = 'purple')
    

m.add_child(choropleths)  
m.add_child(linear)

m

I have not managed to find a way to add graphs directly as popups to choropleths, or to add multiple graphs/text to each popup. However, hacking the input json should make this possible.

Documentation for folium is well developed, and it is even possible to include floating images and videos in map visualisations!

In particular, a great repository of Folium notebooks can be found here, demonstrating the capabilities of the library.

http://nbviewer.jupyter.org/github/python-visualization/folium/tree/v0.2.1/examples/